In [1]:
!pip install ucimlrepo

from ucimlrepo import fetch_ucirepo

auto_mpg = fetch_ucirepo(id=9)

X = auto_mpg.data.features
y = auto_mpg.data.targets

/home/azureuser/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/azureuser/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [2]:
print(X.head())
print(y.head())

   displacement  cylinders  horsepower  weight  acceleration  model_year  \
0         307.0          8       130.0    3504          12.0          70   
1         350.0          8       165.0    3693          11.5          70   
2         318.0          8       150.0    3436          11.0          70   
3         304.0          8       150.0    3433          12.0          70   
4         302.0          8       140.0    3449          10.5          70   

   origin  
0       1  
1       1  
2       1  
3       1  
4       1  
    mpg
0  18.0
1  15.0
2  18.0
3  16.0
4  17.0


In [3]:
print(X.isnull().sum())

displacement    0
cylinders       0
horsepower      6
weight          0
acceleration    0
model_year      0
origin          0
dtype: int64


In [4]:
mean = X['horsepower'].mean()
X['horsepower'].fillna(mean, inplace=True)
print(X.isnull().sum())

displacement    0
cylinders       0
horsepower      0
weight          0
acceleration    0
model_year      0
origin          0
dtype: int64


/tmp/ipykernel_1740/3914048846.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X['horsepower'].fillna(mean, inplace=True)
/tmp/ipykernel_1740/3914048846.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['horsepower'].fillna(mean, inplace=True)


In [5]:
import pandas as pd
data = pd.DataFrame(X, columns=auto_mpg.features)
data['mpg'] = y
display(data.describe())

,displacement,cylinders,horsepower,weight,acceleration,model_year,origin,mpg
count,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000
mean,193.425879,5.454774,104.469388,2970.424623,15.568090,76.010050,1.572864,23.514573
std,104.269838,1.701004,38.199187,846.841774,2.757689,3.697627,0.802055,7.815984
min,68.000000,3.000000,46.000000,1613.000000,8.000000,70.000000,1.000000,9.000000
25%,104.250000,4.000000,76.000000,2223.750000,13.825000,73.000000,1.000000,17.500000
50%,148.500000,4.000000,95.000000,2803.500000,15.500000,76.000000,1.000000,23.000000
75%,262.000000,8.000000,125.000000,3608.000000,17.175000,79.000000,2.000000,29.000000
max,455.000000,8.000000,230.000000,5140.000000,24.800000,82.000000,3.000000,46.600000


In [6]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [10]:
import mlflow
import mlflow.sklearn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from mlflow.models import infer_signature
import numpy as np

# Assuming X_train, X_test, y_train, y_test are already loaded here

mlflow.set_tracking_uri("http://172.178.67.2:5000")
mlflow.set_experiment("capstone")

with mlflow.start_run():
    model = LinearRegression()
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)

    # Calculate metrics
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    mae = mean_absolute_error(y_test, predictions)

    # Log metrics
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("mae", mae)

    # Log parameters
    mlflow.log_param("model_type", "LinearRegression")

    # Infer signature
    signature = infer_signature(X_train, model.predict(X_train))

    # Log the model with registry and example input
    model_info = mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="MPGCompareModel",
        signature=signature,
        input_example=X_train,
        registered_model_name="MPGComparison"
    )

    print("Linear Regression → RMSE:", rmse, " MAE:", mae)


/home/azureuser/anaconda3/lib/python3.9/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/08/13 12:35:00 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
Registered model 'MPGComparison' already exists. Creating a new version of this model...
2025/08/13 12:35:02 INFO mlflow.store.

Linear Regression → RMSE: 2.9964118609531942  MAE: 2.323901950356438
🏃 View run judicious-newt-431 at: http://172.178.67.2:5000/#/experiments/2/runs/e755ff77c0bf43aa8be05885c6e1c0b4
🧪 View experiment at: http://172.178.67.2:5000/#/experiments/2


Created version '2' of model 'MPGComparison'.


In [12]:
import mlflow
import mlflow.sklearn
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from mlflow.models import infer_signature
import numpy as np

# Assuming X_train, X_test, y_train, y_test are already loaded here

mlflow.set_tracking_uri("http://172.178.67.2:5000")
mlflow.set_experiment("capstone")

with mlflow.start_run():
    model = DecisionTreeRegressor()
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    mae = mean_absolute_error(y_test, predictions)

    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("mae", mae)
    mlflow.log_param("model_type", "DecisionTreeRegressor")

    signature = infer_signature(X_train, model.predict(X_train))

    model_info = mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="MPGCompareModel",
        signature=signature,
        input_example=X_train,
        registered_model_name="MPGComparison"
    )

    print("Decision Tree → RMSE:", rmse, " MAE:", mae)


/home/azureuser/anaconda3/lib/python3.9/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/08/13 12:36:46 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
Registered model 'MPGComparison' already exists. Creating a new version of this model...
2025/08/13 12:36:49 INFO mlflow.store.

Decision Tree → RMSE: 3.5040119862808687  MAE: 2.3830000000000005
🏃 View run overjoyed-carp-503 at: http://172.178.67.2:5000/#/experiments/2/runs/320346ba15bb4d148640538d462b3039
🧪 View experiment at: http://172.178.67.2:5000/#/experiments/2


Created version '3' of model 'MPGComparison'.


In [14]:
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from mlflow.models import infer_signature
import numpy as np

# Assuming X_train, X_test, y_train, y_test are already loaded here

mlflow.set_tracking_uri("http://172.178.67.2:5000")
mlflow.set_experiment("capstone")

with mlflow.start_run():
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    mae = mean_absolute_error(y_test, predictions)

    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("mae", mae)
    mlflow.log_param("model_type", "RandomForestRegressor")

    signature = infer_signature(X_train, model.predict(X_train))

    model_info = mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="MPGCompareModel",
        signature=signature,
        input_example=X_train,
        registered_model_name="MPGComparison"
    )

    print("Random Forest → RMSE:", rmse, " MAE:", mae)


/home/azureuser/anaconda3/lib/python3.9/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/home/azureuser/anaconda3/lib/python3.9/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-

Random Forest → RMSE: 2.67029418416773  MAE: 1.7778700000000003
🏃 View run charming-lynx-954 at: http://172.178.67.2:5000/#/experiments/2/runs/94811291886e44179ae045a9df56e748
🧪 View experiment at: http://172.178.67.2:5000/#/experiments/2


Created version '5' of model 'MPGComparison'.


In [15]:
import mlflow
import mlflow.sklearn
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from mlflow.models import infer_signature
import numpy as np

# Assuming X_train, X_test, y_train, y_test are already loaded here

mlflow.set_tracking_uri("http://172.178.67.2:5000")
mlflow.set_experiment("capstone")

with mlflow.start_run():
    model = GradientBoostingRegressor()
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    mae = mean_absolute_error(y_test, predictions)

    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("mae", mae)
    mlflow.log_param("model_type", "GradientBoostingRegressor")

    signature = infer_signature(X_train, model.predict(X_train))

    model_info = mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="MPGCompareModel",
        signature=signature,
        input_example=X_train,
        registered_model_name="MPGComparison"
    )

    print("Gradient Boosting → RMSE:", rmse, " MAE:", mae)


/home/azureuser/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:672: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?
/home/azureuser/anaconda3/lib/python3.9/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handl

Gradient Boosting → RMSE: 2.626412885950547  MAE: 1.8903462807955846
🏃 View run delicate-fowl-302 at: http://172.178.67.2:5000/#/experiments/2/runs/4517b7a7175c456184e69f2fb92aa1be
🧪 View experiment at: http://172.178.67.2:5000/#/experiments/2


Created version '6' of model 'MPGComparison'.


In [17]:
pip install Xgboost


     |████████████████████████████████| 223.5 MB 91.9 MB/s eta 0:00:01     |████████████████████████████████| 223.6 MB 4.3 kB/s  eta 0:00:01
     |████████████████████████████████| 322.5 MB 17 kB/s s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [18]:
import mlflow
import mlflow.sklearn
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from mlflow.models import infer_signature
import numpy as np

# Assuming X_train, X_test, y_train, y_test are already loaded here

mlflow.set_tracking_uri("http://172.178.67.2:5000")
mlflow.set_experiment("capstone")

with mlflow.start_run():
    model = XGBRegressor()
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    mae = mean_absolute_error(y_test, predictions)

    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("mae", mae)
    mlflow.log_param("model_type", "XGBRegressor")

    signature = infer_signature(X_train, model.predict(X_train))

    model_info = mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="MPGCompareModel",
        signature=signature,
        input_example=X_train,
        registered_model_name="MPGComparison"
    )

    print("XGBoost → RMSE:", rmse, " MAE:", mae)


/home/azureuser/anaconda3/lib/python3.9/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/08/13 12:42:32 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
Registered model 'MPGComparison' already exists. Creating a new version of this model...
2025/08/13 12:42:34 INFO mlflow.store.

XGBoost → RMSE: 2.936132721554246  MAE: 1.983784794807434
🏃 View run abrasive-sheep-171 at: http://172.178.67.2:5000/#/experiments/2/runs/212123865e71474ea5322c110c086c82
🧪 View experiment at: http://172.178.67.2:5000/#/experiments/2


Created version '7' of model 'MPGComparison'.


In [1]:
import mlflow
import pickle
 
# Connect to your MLflow tracking server
mlflow.set_tracking_uri("http://172.178.67.2:5000")
 
# Registered model details
reg_model_name = "MPGComparison"
model_version = 5
model_uri = f"models:/{reg_model_name}/{model_version}"
 
# Load the model from MLflow
loaded_model = mlflow.sklearn.load_model(model_uri)
print(f"Loaded model: {reg_model_name} v{model_version}")
 
# Save to local file as lr_model.bin
with open("lr_model.bin", "wb") as f_out:
    pickle.dump(loaded_model, f_out)
 
print("Model saved locally as lr_model.bin")

/home/azureuser/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/azureuser/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


Loaded model: MPGComparison v5
Model saved locally as lr_model.bin
